In [1]:
# !pip install pymongo
# !pip install termcolor

In [2]:
from pymongo import MongoClient, DESCENDING
from pathlib import Path
import datetime
# import json
import simplejson
from pydash import omit, find_index, merge, group_by
from functools import partial
from bson import json_util
import os
import json

from termcolor import colored
from IPython.display import display, Image, JSON
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Box, Button, ButtonStyle, AppLayout, Layout, Style

from lib.PersistentSet import PersistentSet

In [3]:
images_dir = Path('../images')
gallery_dir = Path('../')
handmade_dir = Path('./handmade')
label_dir = handmade_dir/'labels'
label_dir.mkdir(exist_ok=True)
current_ids_filepath = Path('./tmp/labeling_current_ids.json')

In [4]:
mongo = MongoClient('172.17.0.1', 27017)
db = mongo['bad-vis']
posts = db['posts']
imagemeta = db['imagemeta']
imagelabel = db['imagelabel']
vislabels = db['vislabels']

In [5]:
image_url_prefix = 'http://vpn2d.mydev:7000'
# image_url_prefix = 'http://localhost:7000'

# Backup labels

In [6]:
if os.path.isfile(label_dir/'labels.json'):
    os.rename(label_dir/'labels.json', label_dir/f"labels_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.json")

In [7]:
json.dump([omit(l, '_id') for l in vislabels.find()], open(label_dir/'labels.json', 'w'), default=json_util.default)

# Labeling

In [8]:
labelbox_height = 1200
gallery_limit = 50
gallery_height = 1000

all_images = [i for i in imagelabel.find().sort('popularity_score', DESCENDING)]

labelOptions = json.load(open('tmp/labelOptions.json'))
labelOptions = {c['tag']:c['options'] for c in labelOptions}

labelling_states = {
    'filter_states': {
        'tags': set(),
        'offset': 0,
        'image_id': ''
    },
    'all_images': all_images,
    'images': all_images[:50],
    'current_image_id': '',
    'labelOptions': labelOptions
}

current_ids = ''
if not os.path.isfile(current_ids_filepath):
    current_ids = PersistentSet()
    current_ids.set_file(current_ids_filepath)
else:
    current_ids = PersistentSet.load_set(current_ids_filepath)

def set_current_image_id (image_id):
    current_ids.persist_remove(list(current_ids)[0])
    current_ids.persist_add(image_id)

def get_current_image_id ():
    return list(current_ids)[0]

labelBoxOutput = widgets.Output(layout=Layout(height=f"{labelbox_height+30}px"))

In [9]:
# JSON(labelOptions)

In [10]:
remarks_output = widgets.Output()

In [11]:
remarks_output

Output()

In [12]:
def find_image_idx (images, image_id):
    return find_index(images, lambda x: x['image_id'] == image_id)

def find_post (image_id):
    return posts.find_one({'post_id': imagemeta.find_one({'image_id': image_id})['post_id']})

def find_label (image_id):
    image = imagelabel.find_one({'image_id': image_id})
    if not image:
        return None
    image_ids = [image_id] + [i for i in image['duplicated_images']]
    visLabels = [l for l in vislabels.find({'image_id': {'$in': image_ids}})]
    if len(visLabels) > 1:
        print(f"labels warning: more than 1 label {image_ids} {visLabels}")
    return visLabels[0] if visLabels else None

def find_image (image_id):
    if '/' not in image_id:
        image_id = imagemeta.find_one({'short_image_id': image_id}, {'image_id': True})['image_id']
    image = imagelabel.find_one({'$or': [{'image_id': image_id}, {'duplicated_images': image_id}]})
    if not image:
        print(f"Image not found: {image_id}")
    return image

def find_images_by (tags=[], image_id='', offset=0, limit=50):
    if image_id != '':
        image = find_image(image_id)
        return [image] if image else []

    if len(tags) == 0:
        return labelling_states['all_images'][offset:offset+limit]

    images = []
    for l in vislabels.find({'labels': {'$all': tags}}).limit(limit).skip(offset):
        image = find_image(l['image_id'])
        if image:
            images.append(image)
#             images.append({**image, 'labels': l['labels']})
    images.sort(key=lambda x: x['popularity_score'])
    images.reverse()
    return images

## Image Box

In [13]:
def make_image_box (image, width=600, height=800, layoutArgs={}):
#     image = widgets.Image(value=open(gallery_dir/image['labelling_path'], 'rb').read(), width=600, layout=Layout(object_position='center center', object_fit='contain'))
    image = widgets.Image(value=open(gallery_dir/image['labelling_path'], 'rb').read(), width=600, height=height, layout=Layout(object_position='center center', object_fit='contain'))
    boxItems = {
        'image': image
    }

    image_box_layout = Layout(justify_content='center', **layoutArgs)
    image_box_layout = Layout(height=f'{labelbox_height-500}px', justify_content='center', **layoutArgs)
    imageBox = HBox([image], layout=image_box_layout)

    return (imageBox, boxItems)

## Info Box

In [14]:
def make_link (url, text):
    return widgets.HTML(value=f"<a href='{url}' target='_blank'>{text}</a>")

def make_info_box (image, layoutArgs={}):
    output = widgets.Output()

    link = make_link(find_post(image['image_id'])['url'], image['image_id'])
    dp_links = [
        make_link(find_post(dp_image_id)['url'], dp_image_id)
        for dp_image_id in image['duplicated_images']
    ]
    image_link = make_link(f"{image_url_prefix}/projects/bad-vis-images/{image['labelling_path']}", image['labelling_path'])

    description = widgets.Output(layout=Layout(**layoutArgs))
    with description:
        image_idx_in_all_images = find_image_idx(labelling_states['all_images'], image['image_id'])
        len_all_images = len(labelling_states['all_images'])
        print(f"{image_idx_in_all_images} / {len_all_images}")
        print(f"popularity_score: {image['popularity_score']}")
        print(f"phash: {image['phash']}")
        print(f"tags: {len(image['tags'])}")
        for t in image['tags']:
            print(t)

    currentLabelsOutput = widgets.Output(layout=Layout(**layoutArgs))

    outputs = {
        'description': description,
        'currentLabelsOutput': currentLabelsOutput,
        'output': output
    }

    infoBox = VBox([description, currentLabelsOutput, output, link, *dp_links, image_link])

    return (infoBox, outputs)

## Checkboxes

In [15]:
def ckb_box_layout (rows, cols, width):
    return Layout(
        flex_flow='column wrap',
        height=f"{rows*32}px",
        width=f"{cols*(width + 5)}px",
        align_content='flex-start'
    )

def make_ckbs (title, options, n_rows=10, layoutArgs={}, on_ckb_change=None, grouping=False, sorting=True, ckb_width=100):
    ckb_layout = Layout(
        width=f"{ckb_width}px"
    )

    layout = Layout(**layoutArgs)
    border_style = layoutArgs.get('border', '3px solid lightblue')
    layout.border = border_style.replace(border_style.split(' ')[-1], 'orange')

    form = ''
    subcats = []
    ckbsByName = {}

    def ckb_handler (change):
        ckb = change['owner']
        if change['new']:
            ckb.icon = 'check'
            ckb.button_style = 'success'
        else:
            ckb.icon = ''
            ckb.button_style = ''
        if on_ckb_change:
            on_ckb_change(change)
        if len([v for k, v in ckbsByName.items() if v.value]) > 0:
            form.layout.border = form.layout.border.replace(form.layout.border.split(' ')[-1], 'lightblue')
        else:
            form.layout.border = form.layout.border.replace(form.layout.border.split(' ')[-1], 'orange')

    grouped = group_by(options, lambda o: o.get('subcategory', '')) if grouping else {'': options}
    for subcategory, options in grouped.items():
        ckbs = []
        ordered_options = reversed(sorted(options, key=lambda x: x['count'])) if sorting else options
        for n in ordered_options:
            name = n['tag']
            ckb = widgets.ToggleButton(value=False, description=n['name'], tooltip=name, icon='', button_style='', disabled=False, indent=False, layout=ckb_layout)
            ckb.observe(ckb_handler, names='value')
            ckbs.append(ckb)
            ckbsByName[name] = ckb
        subcats.append(VBox(ckbs, layout=ckb_box_layout(rows=n_rows, cols=(-(len(ckbs) // -n_rows)), width=ckb_width)))

    form = VBox([
#         widgets.Label(value=f"{title.capitalize()}"),
        HBox(subcats)
    ], layout=layout)

    return (form, ckbsByName)

def make_remarks (n_rows=10, layoutArgs={}):
    form = ''
    remarks = ''

    layout = Layout(**layoutArgs)
    border_style = layoutArgs.get('border', '3px solid lightblue')
    layout.border = border_style.replace(border_style.split(' ')[-1], 'orange')

    def remarks_handler (change):
        if remarks.value and remarks.value != '':
            form.layout.border = form.layout.border.replace(form.layout.border.split(' ')[-1], 'lightblue')
        else:
            form.layout.border = form.layout.border.replace(form.layout.border.split(' ')[-1], 'orange')

    remarks = widgets.Textarea(value='', layout=Layout(height=f"{n_rows*30-20}px", width=f'240px'), disabled=False)
    remarks.observe(remarks_handler, names='value')
    form = VBox([widgets.Label(value=f"{'remarks'.capitalize()}"), remarks], layout=layout)
    return (form, remarks)

def make_ckb_box (options, n_rows=10, layoutArgs={}, on_ckb_change=None):

    formCkbsBox, formCkbsByName = make_ckbs('form', options['form'], 7, layoutArgs, on_ckb_change, grouping=True, ckb_width=140)
    layoutCkbsBox, layoutCkbsByName = make_ckbs('layout', options['layout'], 7, layoutArgs, on_ckb_change)
    dataCkbsBox, dataCkbsByName = make_ckbs('data', options['data'], n_rows, layoutArgs, on_ckb_change, sorting=False)
    domainCkbsBox, domainCkbsByName = make_ckbs('domain', options['domain'], n_rows, layoutArgs, on_ckb_change)
    mediaCkbsBox, mediaCkbsByName = make_ckbs('media', options['media'], n_rows, layoutArgs, on_ckb_change)
    flagCkbsBox, flagCkbsByName = make_ckbs('flag', options['flag'], n_rows, layoutArgs, on_ckb_change)

    faultCkbsBox, faultCkbsByName = make_ckbs('fault', options['fault'], 10, layoutArgs, on_ckb_change, grouping=True, sorting=False,  ckb_width=180)
    effectCkbsBox, effectCkbsByName = make_ckbs('effect', options['effect'], n_rows, layoutArgs, on_ckb_change)

    remarksBox, remarks = make_remarks(n_rows, layoutArgs=layoutArgs)

    boxItemsByName = {}
    for d in [{'remarks': remarks}, formCkbsByName, faultCkbsByName, dataCkbsByName, domainCkbsByName, layoutCkbsByName, effectCkbsByName, mediaCkbsByName, flagCkbsByName]:
        for k, v in d.items():
            boxItemsByName[k] = v

    boxesByName = {
        'remarks': remarksBox,
        'form': formCkbsBox,
        'layout': layoutCkbsBox,
        'data': dataCkbsBox,
        'domain': domainCkbsBox,
        'media': mediaCkbsBox,
        'flag': flagCkbsBox,
        'fault': faultCkbsBox,
        'effect': effectCkbsBox,
    }

    return (VBox([
        HBox([formCkbsBox, layoutCkbsBox]),
        HBox([dataCkbsBox, domainCkbsBox, mediaCkbsBox, flagCkbsBox, effectCkbsBox, remarksBox]),
        HBox([faultCkbsBox]),
    ]), boxItemsByName, boxesByName)

## Buttons

In [16]:
def make_btn_box (on_update=None, on_prev=None, on_next=None):
    btns = []

    if on_prev:
        prevBtn = widgets.Button(description='Prev', button_style='warning', layout=Layout(width='60px'))
        def go_prev (btn):
            if on_prev:
                on_prev()
        prevBtn.on_click(go_prev)
        btns.append(prevBtn)

        updateAndPrevBtn = widgets.Button(description='Save -> Prev', button_style='info')
        def update_and_prev (btn):
            if on_update:
                on_update()
            if on_prev:
                on_prev()
        updateAndPrevBtn.on_click(update_and_prev)
        btns.append(updateAndPrevBtn)

    updateBtn = widgets.Button(description='Save', button_style='success')
    def update (btn):
        if on_update:
            on_update()
    updateBtn.on_click(update)
    btns.append(updateBtn)

    if on_next:
        updateAndNextBtn = widgets.Button(description='Save -> Next', button_style='info')
        def update_and_next (btn):
            if on_update:
                on_update()
            if on_next:
                on_next()
        updateAndNextBtn.on_click(update_and_next)
        btns.append(updateAndNextBtn)

        nextBtn = widgets.Button(description='Next', button_style='warning', layout=Layout(width='60px'))
        def go_next (btn):
            if on_next:
                on_next()
        nextBtn.on_click(go_next)
        btns.append(nextBtn)

    btnItems = {
        'updateBtn': updateBtn,
        'updateAndPrevBtn': updateAndPrevBtn,
        'updateAndNextBtn': updateAndNextBtn
    }

    return (HBox(btns, layout=Layout(height='34px')), btnItems)

## Label Box

In [17]:
def make_label_box (image_id, options, next_callback=None, prev_callback=None):
    visImage = imagelabel.find_one({'image_id': image_id})
    if not visImage:
        print(f"Not found: {image_id}")
    vislabel = find_label(image_id)

    labels = set(vislabel['labels']) if vislabel else set()

    layoutArgs = {
        'padding': '10px',
        'margin': '5px',
        'border': '3px solid lightblue'
    }

    imageBox, imageBoxItems = make_image_box(visImage, width=600, height=labelbox_height, layoutArgs=layoutArgs)

    infoBox, infoBoxItems = make_info_box(visImage, layoutArgs=layoutArgs)

    invalidLabels = set()
    def show_current_labels ():
        currentLabelsOutput = infoBoxItems['currentLabelsOutput']
        currentLabelsOutput.clear_output()
        with currentLabelsOutput:
            for l in sorted(list(labels)):
                if l in boxItemsByName:
                    print(l)
                else:
                    invalidLabels.add(l)
                    print(colored(l, 'red'))

    def on_ckb_change (change):
        ckb = change['owner']
        labelName = ckb.tooltip
        if change['new']:
            labels.add(labelName)
        else:
            labels.remove(labelName)
        show_current_labels()

    ckbBox, boxItemsByName, boxesByName = make_ckb_box(options, n_rows=7, layoutArgs=layoutArgs, on_ckb_change=on_ckb_change)

    remarks = boxItemsByName['remarks']

    def update_label ():
        vislabels.find_one_and_update({'image_id': vislabel['image_id'] if vislabel else visImage['image_id']}, {
            '$set': {
                'phash': visImage['phash'],
                'labels': sorted(list(labels - invalidLabels)),
                'remarks': remarks.value,
                'updatedAt': datetime.datetime.utcnow()
            },
            '$setOnInsert': {'createdAt': datetime.datetime.utcnow()}
        }, upsert=True)
        with remarks_output:
            if remarks.value:
                print(f"{remarks.value}")
        with infoBoxItems['output']:
            print(f"{visImage['short_image_id']} updated")
        show_current_labels()

    def on_update ():
        update_label()

    def on_prev ():
        if prev_callback:
            prev_callback(image_id)

    def on_next ():
        if next_callback:
            next_callback(image_id)

    btnBox, btnBoxItems = make_btn_box(on_update=on_update, on_prev=on_prev, on_next=on_next)

    panelBoxLayout = Layout(
        justify_content= 'flex-start',
        align_items= 'flex-start',
        align_content= 'flex-start',
    )
    panelBox = VBox([ckbBox, btnBox], layout=panelBoxLayout)

    for l in labels:
        if l in boxItemsByName:
            boxItemsByName[l].value = True

    remarks.value = vislabel['remarks'] if vislabel else ''

    with infoBoxItems['output']:
        print('ready')

    leftBox = VBox([imageBox, infoBox])

    pane_widths = [2, 4, 1]
    return AppLayout(left_sidebar=leftBox, center=panelBox, pane_widths=pane_widths, height=f'{labelbox_height+10}px')

In [18]:
def next_id (image_id, prev=False):
    images = labelling_states['all_images'] if len(labelling_states['filter_states']['tags']) == 0 else labelling_states['images']
    idx = find_image_idx(images, image_id)
    new_idx = max(0, idx - 1) if prev else min(len(images) - 1, idx + 1)
    new_image_id = images[new_idx]['image_id']
    set_current_image_id(new_image_id)
    return new_image_id

def prev_img (output, image_id):
    show_img(next_id(image_id, prev=True), output)

def next_img (output, image_id):
    show_img(next_id(image_id, prev=False), output)

def show_img (image_id, output=None):
    output.clear_output()
    with output:
        display(make_label_box(image_id, options=labelOptions, next_callback=partial(next_img, output), prev_callback=partial(prev_img, output)))

## Gallery Box

In [19]:
def make_filter_box (options, on_filter_change=None, layoutArgs=None):
    layout = Layout(**layoutArgs)

    def ckb_handler (change):
        tag = change['owner'].tag
        if change['new']:
            labelling_states['filter_states']['tags'].add(tag)
        else:
            labelling_states['filter_states']['tags'].remove(tag)
        if on_filter_change:
            on_filter_change(labelling_states['filter_states'])

    def image_id_textbox_handler (change):
        labelling_states['filter_states']['image_id'] = change['new']
        on_filter_change(labelling_states['filter_states'])

    def offset_textbox_handler (change):
        labelling_states['filter_states']['offset'] = change['new']
        on_filter_change(labelling_states['filter_states'])

    ckbs = []
    image_id_textbox = widgets.Text(value='', placeholder='1yb04c_0', description='Image ID:', disabled=False, layout={'width':'180px'})
    image_id_textbox.observe(image_id_textbox_handler, names='value')
    ckbs.append(image_id_textbox)
    offset_textbox = widgets.BoundedIntText(value=labelling_states['filter_states']['offset'], min=0, max=len(labelling_states['all_images']), step=gallery_limit, description='Offset', disabled=False, layout={'width':'180px'})
    offset_textbox.observe(offset_textbox_handler, names='value')
    ckbs.append(offset_textbox)
    for cat, opts in options.items():
        for subcat, options in group_by(opts, lambda o: o.get('subcategory', '')).items():
            ckbs.append(widgets.widgets.HTML(value=f"<b>{cat.upper()}</b> {subcat.capitalize()}"))
            for o in options:
                ckb = widgets.Checkbox(value=False, description=f"{o['name']} ({o['count']})", disabled=False, indent=False)
                ckb.tag = o['tag']
                ckb.observe(ckb_handler, names='value')
                ckbs.append(ckb)

    return VBox(ckbs, layout=layout)

def make_gallery_box (images, on_select=None, layoutArgs=None):

    def click_handler (btn):
        image_id = btn.tooltip
        if on_select:
            on_select(image_id)

    image_boxes = []
    for i in images:
        image_box = widgets.Image(value=open(f"../{i['labelling_thumbnail_path']}", 'rb').read(), width=140, height=140, layout=Layout(width='140px', height='140px'))
        btn = widgets.Button(description=f"{i['image_name']} ({i['popularity_score']})", tooltip=i['image_id'])
        btn.on_click(click_handler)
        image_boxes.append(VBox([image_box, btn]))

    return Box(image_boxes,
               layout=Layout(display='flex', flex_flow='row wrap'))

def make_gallery (options, select_callback=None):
    layoutArgs = {
        'padding': '10px',
        'margin': '5px',
        'border': '3px solid lightblue'
    }

    def select_handler (image_id):
        show_img(image_id, labelBoxOutput)

    labelling_states['images'] = find_images_by(limit=gallery_limit, offset=labelling_states['filter_states']['offset'])

    galleryOutput = widgets.Output(layout=Layout(**layoutArgs, height=f"{gallery_height-20}px"))
    with galleryOutput:
        display(make_gallery_box(labelling_states['images'], on_select=select_handler, layoutArgs=layoutArgs))

    def filter_handler (filter_states):
        labelling_states['images'] = find_images_by(tags=list(filter_states['tags']), image_id=filter_states['image_id'], offset=filter_states['offset'], limit=gallery_limit)
        galleryOutput.clear_output()
        with galleryOutput:
            display(make_gallery_box(labelling_states['images'], on_select=select_handler, layoutArgs=layoutArgs))

    filterBox = make_filter_box(options, on_filter_change=filter_handler, layoutArgs=layoutArgs)

    pane_widths = [1, 5, 1]
    return AppLayout(left_sidebar=filterBox, center=galleryOutput, pane_widths=pane_widths, height=f"{gallery_height}px")

# GUI

In [20]:
make_gallery(labelling_states['labelOptions'])

AppLayout(children=(VBox(children=(Text(value='', description='Image ID:', layout=Layout(width='180px'), place…

In [21]:
labelBoxOutput

Output(layout=Layout(height='1230px'))

In [22]:
show_img(get_current_image_id(), labelBoxOutput)

# Maintenance

In [23]:
# for label in vislabels.find():
#     visimage = imagelabel.find_one({'image_id': label['image_id']})
#     if not visimage:
# #         print(f"missing {label['image_id']}")
#         duplicated = [i for i in imagelabel.find({'duplicated_images': label['image_id']})]
#         if len(duplicated) > 1:
#             pass
# #             print(f"found more than one duplicated {label['image_id']} {duplicated}")
#         if not duplicated:
# #             pass
#             print(f"missing in duplicated either {label['image_id']}")
#             print(json.dumps(omit(label, '_id'), default=datetime.datetime.isoformat))
#         else:
#             duplicated = duplicated[0]
# #             print(f"found in duplicated {label['image_id']} {duplicated['image_id']}")

In [24]:
# for label in vislabels.find():
#     if 'tumblr' in label['image_id']:
#         continue

#     visimage = imagelabel.find_one({'image_id': label['image_id']})
#     if not visimage:
#         visimage = imagelabel.find_one({'duplicated_images': label['image_id']})

#     if not visimage:
#         print(f"missing {label['image_id']}")
#     else:
#         label['image_id'] = visimage['image_id']
#         label['phash'] = visimage['phash']
#         vislabels.update_one({'_id': label['_id']}, {'$set': {'image_id': label['image_id'], 'phash': label['phash']}})

In [25]:
# # replace labels
# current_label = 'fault:percentage'
# new_label = ''

# for l in vislabels.find():
#     if len([l for l in l['labels'] if current_label == l]) > 0:
#         ori_length = len(l['labels'])
#         print(len(l['labels']), l['_id'], l['image_id'], l['labels'])

#         l['labels'] = [l for l in l['labels'] if current_label != l]
# #         if new_label not in l['labels']:
# #             l['labels'].append(new_label)

#         new_length = len(l['labels'])
#         print(new_length - ori_length, new_label in l['labels'], len(l['labels']), l['labels'])

# #         vislabels.update_one({'_id': l['_id']}, {'$set': {'labels': l['labels']}})
# #     break